---------------------------------------------------------------------------------------------------------------------------------

##### <span style="color: #FF0000;">Weekly Tasks - Support Vector Machines</span>

##### Tasks:
- Implement SVM classification on the iris dataset using different approaches
- Compare performance between linear and non-linear SVM models

##### Task Details:
1. Linear SVM Implementation:
   - Use LinearSVC to separate virginica from non-virginica
   - Evaluate model performance using standard metrics
   - Experiment with different C values for regularization
   - Analyze decision boundaries and support vectors

2. Non-linear SVM Exploration:
   - Implement polynomial kernel SVM
   - Test different kernel options (polynomial, RBF)
   - Compare performance with linear SVM
   - Visualize decision boundaries for different kernels

3. Multi-class Classification (Optional):
   - Extend the implementation to classify all three iris species
   - Compare one-vs-rest and one-vs-one approaches
   - Evaluate overall model performance
   - Analyze confusion matrix for insights

##### Data Preparation Note:
For details on the data preparation process, please refer to the data folder at **SVM\Weekly Tasks\data**. The iris dataset will be used for this analysis, which contains measurements for 150 iris flowers from three different species. Each record includes four features: sepal length, sepal width, petal length, and petal width. This dataset is ideal for demonstrating SVM concepts as it provides clear opportunities to explore both linear and non-linear classification scenarios.

##### <span style="color: #1E90FF;">[Click here to jump to the main solution](#model-development-and-evaluation)</span>

---------------------------------------------------------------------------------------------------------------------------------

In [1]:
# Install required libraries
%pip install pandas numpy matplotlib seaborn scikit-learn wordcloud xgboost statsmodels 
%pip install shap lime 

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


---------------------------------------------------------------------------------------------------------------------------------

##### <span style="color: #FF0000;">Import Required Libraries</span>
##### This code imports the required libraries for data manipulation, visualization, and machine learning tasks.

In [2]:
#Import required libraries
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import multiprocessing
from collections import Counter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

import sys

assert sys.version_info >= (3, 7)

import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)


---------------------------------------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------------------------------------

### Model Building-Main